# Import modules

In [2]:
    %reload_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np

pd.options.display.max_columns = 999
pd.options.display.max_rows = 1999
# pd.set_option('display.max_colwidth', -1)

import warnings
warnings.filterwarnings('ignore')

import os
import sys
sys.path.insert(0, '../')
from scpts import alziras

import basedosdados as bd

out_path = '../data/analise/mais/alziras/output/'
in_path  = '../data/analise/mais/alziras/input/'

### GERA DECIL

In [20]:
query = """
  SELECT
    SAFE_CAST(
      REPLACE(SAFE_CAST(t1.ano AS STRING),"2018","2020")
      AS INT64
    ) as ano,
    t2.ano as ano_2,
    t1.pib,
    t2.populacao,
    t1.id_municipio,
    IFNULL(SAFE_DIVIDE(t1.pib,t2.populacao),0) as pib_per_capta
  FROM `basedosdados.br_ibge_pib.municipio` t1
  JOIN `basedosdados.br_ibge_populacao.municipio` t2
    ON t1.id_municipio = t2.id_municipio and t1.ano = t2.ano
  WHERE t1.ano=2018 or t1.ano=2016

"""
pib = bd.read_sql(query, billing_project_id='basedosdados-dev')

Downloading: 100%|█████████████████████████████████████████| 11140/11140 [00:01<00:00, 9367.47rows/s]


In [28]:
pib['decil'] = pd.qcut(pib["pib_per_capta"], 10, labels=False) + 1

In [29]:
pib = pib.sort_values(by='pib_per_capta')

In [47]:
lower = pib.drop_duplicates('decil', keep='first')
upper = pib.drop_duplicates('decil', keep='last')

In [61]:
itens = zip(upper['pib_per_capta'].tolist(),upper['decil'].tolist(),
           lower['pib_per_capta'].tolist(),lower['decil'].tolist())
for u_pib,u_decil,l_pib,l_decil in itens:
    print(f"WHEN t2.pib_per_capita >={round(l_pib,0)} and t2.pib_per_capita <{round(u_pib,0)} THEN {u_decil}")

WHEN t2.pib_per_capita >=0.0 and t2.pib_per_capita <7288.0 THEN 1
WHEN t2.pib_per_capita >=7290.0 and t2.pib_per_capita <8684.0 THEN 2
WHEN t2.pib_per_capita >=8684.0 and t2.pib_per_capita <10452.0 THEN 3
WHEN t2.pib_per_capita >=10452.0 and t2.pib_per_capita <13149.0 THEN 4
WHEN t2.pib_per_capita >=13153.0 and t2.pib_per_capita <16597.0 THEN 5
WHEN t2.pib_per_capita >=16598.0 and t2.pib_per_capita <20395.0 THEN 6
WHEN t2.pib_per_capita >=20402.0 and t2.pib_per_capita <24834.0 THEN 7
WHEN t2.pib_per_capita >=24839.0 and t2.pib_per_capita <30817.0 THEN 8
WHEN t2.pib_per_capita >=30828.0 and t2.pib_per_capita <42042.0 THEN 9
WHEN t2.pib_per_capita >=42065.0 and t2.pib_per_capita <583172.0 THEN 10


### SALVA TABELAS COMPLETAS

In [89]:
query = """
SELECT * FROM `basedosdados-projetos.alziras_eleicoes.prefeitos_completo`
"""
prefeitos_completo = bd.read_sql(query, billing_project_id='basedosdados-dev')

Downloading: 100%|█████████████████████████████████████████| 34880/34880 [00:09<00:00, 3826.68rows/s]


In [90]:
prefeitos_completo.head()

,ano,tipo_eleicao,turno,sigla_uf,id_municipio_tse,numero_candidato,id_candidato_bd,cargo,sigla_partido,votos,resultado,sequencial,cpf,nome_urna,numero_partido,situacao,genero,instrucao,idade,raca,faixa_etaria,id_municipio,PIB,populacao,municipio,regiao,pib_per_capta,porte_municipal,decil_pib,nome,reeleito_2020,valor_receita
0,2020,eleicao ordinaria,1,TO,73040,55,859,prefeito,PSD,787,nao eleito,270001118052,00012367176,Evandro,55,deferido,masculino,ensino superior completo,39.0,amarela,30 a 40 anos,1713809,68754151,6570.0,Palmeiras do Tocantins,Norte,10464.863166,0 a 20 mil,4,Evandro Pereira De Sousa,,48214.20
1,2016,eleicao ordinaria,1,SP,71579,45,7991,prefeito,PSDB,67643,eleito,250000044452,00107100860,Fernando Fernandes,45,deferido,masculino,ensino superior completo,65.0,branca,60 mais,3552809,8113750992,275948.0,Taboão da Serra,Sudeste,29403.188253,100 a 500 mil,8,Fernando Fernandes Filho,n,1192034.75
2,2020,eleicao ordinaria,1,MA,8109,45,8631,prefeito,PSDB,981,nao eleito,100000734869,00116006323,Flávio Do Santa Maria,45,deferido,masculino,ensino medio completo,38.0,branca,30 a 40 anos,2109056,40477198,5995.0,Porto Rico do Maranhão,Nordeste,6751.826188,0 a 20 mil,1,Flavio Mendes Da Silva,,NaN
3,2020,eleicao ordinaria,1,MS,91596,14,9622,prefeito,PTB,1700,nao eleito,120000781316,00129929174,Leandro Caramalac,14,deferido,masculino,ensino medio completo,38.0,branca,30 a 40 anos,5008008,515238184,21311.0,Terenos,Centro-Oeste,24177.100277,20 a 50 mil,7,Leandro Guimarães Caramalac Da Costa,,17000.00
4,2016,eleicao ordinaria,1,RJ,60011,15,19389,prefeito,MDB,488775,nao eleito,190000008414,00262905701,Pedro Paulo,15,deferido,masculino,ensino superior completo,44.0,branca,40 a 50 anos,3304557,328440481384,6498837.0,Rio de Janeiro,Sudeste,50538.347305,maior 500 mil,10,Pedro Paulo Carvalho Teixeira,,NaN


In [91]:
prefeitos_completo['idade'] = prefeitos_completo['idade'].apply(lambda x: str(x).split('.')[0])
prefeitos_completo['populacao'] = prefeitos_completo['populacao'].apply(lambda x: str(x).split('.')[0])

In [92]:
prefeitos_completo.to_csv('/Users/m/Downloads/prefeitos_completo.csv', index=False,encoding='utf-8' )

In [93]:
#### vereadores

In [94]:
query = """
SELECT * FROM `basedosdados-projetos.alziras_eleicoes.vereadores_completo`
"""
vereadores_completo = bd.read_sql(query, billing_project_id='basedosdados-dev')

Downloading: 100%|███████████████████████████████████████| 934424/934424 [04:15<00:00, 3651.40rows/s]


In [95]:
vereadores_completo.head()

,ano,tipo_eleicao,turno,sigla_uf,id_municipio_tse,numero_candidato,id_candidato_bd,cargo,sigla_partido,votos,resultado,sequencial,cpf,nome_urna,numero_partido,situacao,genero,instrucao,idade,raca,faixa_etaria,id_municipio,PIB,populacao,municipio,regiao,pib_per_capta,porte_municipal,decil_pib,nome,reeleito_2020,valor_receita
0,2020,eleicao ordinaria,1,RS,89630,17789,29,vereador,PSL,89,nao eleito,210000881997,00000520004,Anelise Goulart,17,deferido,feminino,ensino medio completo,40.0,parda,40 a 50 anos,4323002,3743506026,254101.0,Viamão,Sul,14732.354560,100 a 500 mil,5,Anelise Goulart De Souza,,NaN
1,2020,eleicao ordinaria,1,SE,31275,11111,36,vereador,PP,826,eleito,260000858783,00000602590,Erivaldo De Valmir,11,deferido,masculino,ensino fundamental incompleto,40.0,parda,40 a 50 anos,2801405,216393100,21724.0,Carira,Nordeste,9961.015467,20 a 50 mil,3,José Erivaldo Dos Reis,n,1666.0
2,2020,eleicao ordinaria,1,PA,5258,35678,72,vereador,PMB,8,nao eleito,140000736838,00001153200,Elisangela Lopes,35,deferido,feminino,ensino fundamental completo,32.0,parda,30 a 40 anos,1506302,174981161,23424.0,Salvaterra,Norte,7470.165685,20 a 50 mil,2,Elisangela Da Silva Lopes,,60.0
3,2016,eleicao ordinaria,1,AM,2550,28999,132,vereador,PRTB,137,nao eleito,40000006258,00002128730,Merica Carioca,28,deferido,masculino,ensino fundamental completo,51.0,preta,50 a 60 anos,1302603,70234306331,2094391.0,Manaus,Norte,33534.476767,maior 500 mil,9,Adailton Cardoso Maia,,NaN
4,2020,eleicao ordinaria,1,CE,15873,14777,145,vereador,PTB,61,nao eleito,60001049241,00002312310,Danilo Mendes,14,deferido,masculino,ensino fundamental incompleto,45.0,branca,40 a 50 anos,2300754,435221268,43131.0,Amontada,Nordeste,10090.683453,20 a 50 mil,3,Jose Rodrigues Mendes,,NaN


In [96]:
vereadores_completo['idade'] = vereadores_completo['idade'].apply(lambda x: str(x).split('.')[0])
vereadores_completo['populacao'] = vereadores_completo['populacao'].apply(lambda x: str(x).split('.')[0])

In [97]:
vereadores_completo.to_csv('/Users/m/Downloads/vereadores_completo.csv', index=False,encoding='utf-8' )

### DEBUG

In [145]:
query = """

WITH resultado as (
  SELECT
    t1.ano,
    t1.cargo,
    IFNULL(t2.genero,"nao preenchido") as genero,
    IFNULL(t2.raca,"nao preenchido") as raca,
    REPLACE(
      REPLACE(t1.resultado,r"por qp", "" ), 
      r"por media", "") as resultado,
--     SUM(1) as eleitos,
--     SUM(t1.votos) as votos
    t2.nome_urna,
    t1.id_candidato_bd as id_candidato_bd_t1,
    t2.id_candidato_bd as id_candidato_bd_t2,
    t1.tipo_eleicao as tipo_eleicao_t1,
    t2.tipo_eleicao as tipo_eleicao_t2,
    t1.sigla_uf as sigla_uf_t1,
    t2.sigla_uf as sigla_uf_t2,
    t1.numero_candidato as numero_candidato_t1,
    t2.numero  as numero_candidato_t2,
    t1.cargo as cargo_t1,
    t2.cargo as cargo_t2,
    t1.id_municipio as id_municipio_t1,
    t2.id_municipio as id_municipio_t2,
  FROM `basedosdados.br_tse_eleicoes.resultados_candidato` t1
  JOIN `basedosdados.br_tse_eleicoes.candidatos` t2
  ON t1.ano=t2.ano 
    AND t1.tipo_eleicao = t2.tipo_eleicao 
    AND t1.sigla_uf =t2.sigla_uf 
    AND t1.numero_candidato = t2.numero 
    AND t1.cargo =t2.cargo 
    AND t1.id_municipio =t2.id_municipio 
  WHERE (t1.ano=2016 or t1.ano=2020) 
--     AND (t1.cargo='prefeito' or t1.cargo='vereador' )
    AND (t1.cargo='prefeito')
    AND (t1.resultado LIKE '%eleito%' and t1.resultado != 'nao eleito')
    AND t1.tipo_eleicao = "eleicao ordinaria"
    AND t1.resultado IS NOT NULL
    AND t2.ano = 2016
    AND t2.situacao LIKE '%deferido%'
--   GROUP BY 1,2,3,4,5
  ORDER BY 1,2,3,4
  )
  
select * from resultado 

--   resultado_eleitos as (
--     SELECT
--       t1.ano,
--       t1.cargo,
--       t1.genero,
--       t1.raca,
--       t1.resultado,
--       t1.eleitos,
--     SUM(t1.eleitos) OVER(
--       PARTITION BY t1.ano, t1.cargo,t1.genero
--       ORDER BY t1.ano, t1.cargo,t1.genero
--     ) as eleitos_total,
--     t1.votos,
--     SUM(t1.votos) OVER(
--       PARTITION BY t1.ano, t1.cargo,t1.genero
--       ORDER BY t1.ano, t1.cargo,t1.genero
--     ) as votos_total
--     FROM resultado as t1
--   )
  
-- SELECT
--   t1.ano,
--   t1.cargo,
--   t1.genero,
--   t1.raca,
--   t1.eleitos,
--   t1.eleitos_total,
--   SAFE_DIVIDE(eleitos,eleitos_total) as eleitos_proporcional,
--   t1.votos,
--   t1.votos_total,
--   SAFE_DIVIDE(votos,votos_total) as votos_proporcional,
-- FROM resultado_eleitos as t1
-- ORDER BY 1,2,3,4


"""

df = bd.read_sql(query, billing_project_id='basedosdados-dev')

Downloading: 100%|███████████████████████████████████████████| 5537/5537 [00:01<00:00, 5225.21rows/s]


In [146]:
df['primary_key'] = df['sigla_uf_t1'] + '_' + df['id_municipio_t1'] + '_' + df['cargo_t1'] + '_' + df['numero_candidato_t1']

dd = pd.DataFrame(df['primary_key'].value_counts()).reset_index()
dd = dd[dd['primary_key']==2]
duplicate_mun = dd['index'].tolist()

mask = df['primary_key'].isin(duplicate_mun)
bug = df[mask].sort_values(by='primary_key')[['ano','resultado','nome_urna','primary_key']]

In [147]:

bug

,ano,resultado,nome_urna,primary_key
24,2016,eleito,Maria Oliveira,AM_1301803_prefeito_45
708,2016,eleito,Armando Filho,AM_1301803_prefeito_45
4230,2016,eleito,Orlando Filho,BA_2904704_prefeito_45
4229,2016,eleito,Vinícius De Orlando Filho,BA_2904704_prefeito_45
4262,2016,eleito,Franco De Aldemir,BA_2910602_prefeito_10
4261,2016,eleito,Aldemir,BA_2910602_prefeito_10
41,2016,eleito,Dr. Christine,BA_2911808_prefeito_55
772,2016,eleito,Ademar Pinto,BA_2911808_prefeito_55
5466,2016,eleito,Bolinha,BA_2929750_prefeito_70
58,2016,eleito,Marcia De Bolinha,BA_2929750_prefeito_70


In [143]:
bug.to_csv('/Users/m/Downloads/candidatos_duplicados.csv', index=False)

In [144]:
bug.head()

,ano,resultado,nome_urna,primary_key
4175,2016,eleito,Anderson,AL_2705101_prefeito_55
683,2016,eleito,Marquinhos,AL_2705101_prefeito_55
24,2016,eleito,Maria Oliveira,AM_1301803_prefeito_45
711,2016,eleito,Armando Filho,AM_1301803_prefeito_45
4266,2016,eleito,Orlando Filho,BA_2904704_prefeito_45


In [134]:
https://divulgacandcontas.tse.jus.br/divulga/#/municipios/2016/2/28010/candidatos

In [111]:
mask = (df['id_candidato_bd_t1'].isnull()) | (df['id_candidato_bd_t2'].isnull())

df[mask]

,ano,cargo,genero,raca,resultado,nome_urna,id_candidato_bd_t1,id_candidato_bd_t2,tipo_eleicao_t1,tipo_eleicao_t2,sigla_uf_t1,sigla_uf_t2,numero_candidato_t1,numero_candidato_t2,cargo_t1,cargo_t2,id_municipio_t1,id_municipio_t2
74,2016,prefeito,feminino,branca,eleito,Alizandra,None,192184,eleicao ordinaria,eleicao ordinaria,CE,CE,13,13,prefeito,prefeito,2311207,2311207
117,2016,prefeito,feminino,branca,eleito,Dra. Tatyana Mendes,None,263326,eleicao ordinaria,eleicao ordinaria,MA,MA,65,65,prefeito,prefeito,2109056,2109056
120,2016,prefeito,feminino,branca,eleito,Gilvana Do Ze Mario,None,880407,eleicao ordinaria,eleicao ordinaria,MA,MA,12,12,prefeito,prefeito,2111102,2111102
130,2016,prefeito,feminino,branca,eleito,Marlene Fubá,None,1174425,eleicao ordinaria,eleicao ordinaria,MG,MG,22,22,prefeito,prefeito,3119005,3119005
134,2016,prefeito,feminino,branca,eleito,Mônica,None,1461183,eleicao ordinaria,eleicao ordinaria,MG,MG,14,14,prefeito,prefeito,3120839,3120839
144,2016,prefeito,feminino,branca,eleito,Paula De Diva,None,381798,eleicao ordinaria,eleicao ordinaria,MG,MG,55,55,prefeito,prefeito,3136579,3136579
145,2016,prefeito,feminino,branca,eleito,Dona Diva,None,1678926,eleicao ordinaria,eleicao ordinaria,MG,MG,55,55,prefeito,prefeito,3136579,3136579
206,2016,prefeito,feminino,branca,eleito,Magna Dantas,None,779931,eleicao ordinaria,eleicao ordinaria,PB,PB,15,15,prefeito,prefeito,2506103,2506103
226,2016,prefeito,feminino,branca,eleito,Adriana Assunção,None,1306318,eleicao ordinaria,eleicao ordinaria,PE,PE,40,40,prefeito,prefeito,2605806,2605806
407,2016,prefeito,feminino,branca,eleito,Simone Marquetto,None,854762,eleicao ordinaria,eleicao ordinaria,SP,SP,15,15,prefeito,prefeito,3522307,3522307


### Download do basedosdados

In [2]:
# ##-----------------------------###
# q_municipios = """
# SELECT id_municipio,id_municipio_tse,municipio,regiao
# FROM `basedosdados.br_bd_diretorios_brasil.municipio`
# """
# municipios = bd.read_sql(q_municipios, billing_project_id="basedosdados-dev")
# municipios.to_parquet(f"{in_path}municipios.parquet")

# ## Descomentar para rodar

# q_resultado_candidatos = """
# SELECT * FROM `basedosdados.br_tse_eleicoes.resultados_candidato` 
# where (ano=2016 or ano=2020)
# """
# re_secao = bd.read_sql(q_resultado_candidatos, billing_project_id="basedosdados-dev")
# re_secao.to_parquet(f"{in_path}resultado_candidatos.parquet")

# ###-----------------------------###
# q_candidatos = """
# SELECT * FROM `basedosdados.br_tse_eleicoes.candidatos`
# where (ano=2016 or ano=2020) 
# """
# candidatos_capitais = bd.read_sql(q_candidatos, billing_project_id="basedosdados-dev")
# candidatos_capitais.to_parquet(f"{in_path}candidatos.parquet")

# ##-----------------------------###
# q_municipios = """
# SELECT id_municipio,id_municipio_tse,municipio,regiao
# FROM `basedosdados.br_bd_diretorios_brasil.municipio`
# """
# municipios = bd.read_sql(q_municipios, billing_project_id="basedosdados-dev")
# municipios.to_parquet(f"{in_path}municipios.parquet")

# ##-----------------------------###
# q_municipios_pib = """
# with pib_municipios as (
# select id_municipio,ano,PIB
# from `basedosdados.br_ibge_pib.municipios`
# where(ano=2016 or ano=2018)
# ),

# municipios_pop as (
# select *
# from `basedosdados.br_ibge_populacao.municipios`
# where(ano=2016 or ano=2018)
# )

# select * from pib_municipios
# join municipios_pop on (municipios_pop.id_municipio = pib_municipios.id_municipio) and (municipios_pop.ano = pib_municipios.ano)
# """
# pib_municipios = bd.read_sql(q_municipios_pib, billing_project_id="basedosdados-dev").drop(["id_municipio_1", "ano_1"], 1)
# pib_municipios.to_parquet(f"{in_path}municipios_pib.parquet")

# q_bens_candidato = """
# SELECT * FROM `basedosdados.br_tse_eleicoes.bens_candidato`
# where (ano=2016 or ano=2020)
# """
# besn = bd.read_sql(q_bens_candidato, billing_project_id="basedosdados-dev")
# besn.to_parquet(f"{in_path}bens_candidato.parquet")

# q_receita_candidatos = """
# SELECT * FROM `basedosdados-dev.br_tse_eleicoes.receitas_candidato`
# where (ano=2016 or ano=2020)
# """
# receita = bd.read_sql(q_receita_candidatos, billing_project_id="basedosdados-dev")
# receita.to_parquet(f"{in_path}receita_candidatos.parquet")

# q_despesa_candidatos = """
# SELECT * FROM `basedosdados-dev.br_tse_eleicoes.despesas_candidato`
# where (ano=2016 or ano=2020)
# """
# despesa = bd.read_sql(q_despesa_candidatos, billing_project_id="basedosdados-dev")
# despesa.to_parquet(f"{in_path}despesa_candidatos.parquet")




# Prefeitas e Vereadoras

### Import Data

In [3]:
cadidato_resultado = pd.read_parquet(f"{in_path}resultado_candidatos.parquet")
candidatos = pd.read_parquet(f"{in_path}candidatos.parquet")
municipios = pd.read_parquet(f"{in_path}/municipios.parquet")
pib_municipios = pd.read_parquet(f"{in_path}municipios_pib.parquet")

In [4]:
cadidato_resultado['id_municipio_tse'] = cadidato_resultado['id_municipio_tse'].astype('str')
candidatos['id_municipio_tse'] = candidatos['id_municipio_tse'].astype('str')
pib_municipios['id_municipio'] = pib_municipios['id_municipio'].astype('str')

In [5]:
final_table = alziras.get_final_table(
    cadidato_resultado, candidatos, municipios, pib_municipios
)

In [6]:
# mask = final_table['municipio']=='Manaus'
# final_table[mask]

### Gera Tabelas
-- --

**Definições**
 * Utilizei o PIB de 2018 como a versão mais recente para o calculo do PIB percapita de 2020
 * Utilizei Negros como sendo pretos + pardos
-- --
**Duvidas**
 * 0 a 20 mil - 20 a 30mil ?? não seria 0 a 20 mil - 21 a 30 mil para não repitir o 20 mil. Mesmo vale para faixa etaria
 * Mulheres são todas as mulheres ou todas menos mulheres negras?
 * Proporção de negros em relção ao total de negros daquele atributo ou proporção de negros em relação ao total do genero?
 -- --

**Sugestões**
 * Quebrar por resuldado??
 * Possibilidade de gerar mais atributos de agregação utilizando id_municipio
-- --


### Geral

In [7]:
## padroniza eleitos
final_table["resultado"] = (
    final_table["resultado"]
    .str.replace("eleito por qp", "eleito")
    .str.replace("eleito por media", "eleito")
)

## somente eleitos
mask = final_table["resultado"].isin(["eleito"])
final_table = final_table[mask]

## genero declarado
mask = final_table["genero"].notnull()
final_table = final_table[mask]

## raca declarada
mask = final_table["raca"].notnull()
final_table = final_table[mask]

In [8]:
### Genero
group_cols = ["ano", "cargo", "genero", "raca"]  # + ['sigla_uf']
select_cols = ["contagem", "votos"]
atributo = None

df_atributo, df_total = alziras.group_genero_raca(
    final_table, group_cols, select_cols, atributo
)

df_atributo.to_csv(f"{out_path}contagem_e_votos.csv", index=False)

In [12]:
df_atributo[(df_atributo['cargo'] == 'prefeito') & (df_atributo['genero'] == 'feminino') & (df_atributo['ano'] == 2020)]['eleitos'].sum()

672

### Atributos

In [9]:
### Genero
group_cols = ["ano", "cargo", "genero", "raca"]  # + ['sigla_uf']
select_cols = ["contagem", "votos"]
atributos = [
    "instrucao",
    "faixa_etaria",
    "ocupacao",
    "estado_civil",
    "sigla_uf",
    "regiao",
    "porte_municipal",
    "decil",
    "sigla_partido",
]

for atributo in atributos:

    df_atributo, df_total = alziras.group_genero_raca(
        final_table, group_cols, select_cols, atributo, out_path, save=True
    )
    print(atributo)
    print("quantidade de linhas = ", df_atributo.shape[0])
    print(f"quantidade de {atributo} = ", len(df_atributo[atributo].unique()), "\n")


instrucao
quantidade de linhas =  280
quantidade de instrucao =  7 

faixa_etaria
quantidade de linhas =  200
quantidade de faixa_etaria =  5 

ocupacao
quantidade de linhas =  9240
quantidade de ocupacao =  231 

estado_civil
quantidade de linhas =  160
quantidade de estado_civil =  4 

sigla_uf
quantidade de linhas =  1040
quantidade de sigla_uf =  26 

regiao
quantidade de linhas =  200
quantidade de regiao =  5 

porte_municipal
quantidade de linhas =  200
quantidade de porte_municipal =  5 

decil
quantidade de linhas =  400
quantidade de decil =  10 

sigla_partido
quantidade de linhas =  1320
quantidade de sigla_partido =  33 



# Financeiro

### Import Data

In [10]:
##### load despesa
load_cols_despesa = ["ano", "sequencial_candidato", "valor_despesa"]

despesa = pd.read_parquet(
    f"{in_path}despesa_candidatos.parquet", columns=load_cols_despesa
)

##### load receita
load_cols_receita = [
    "ano",
    "sequencial_candidato",
    "sigla_uf",
    "origem_receita",
    "fonte_receita",
    "valor_receita",
]

receita = pd.read_parquet(
    f"{in_path}receita_candidatos.parquet", columns=load_cols_receita
)

### remove valores preenchidos errado
mask = (receita["sigla_uf"] == "AM") & (receita["valor_receita"] > 10000000)
receita = receita[np.logical_not(mask)]


##### load bens
load_cols_bens = ["ano", "sequencial_candidato", "valor_item"]

bens = pd.read_parquet(f"{in_path}bens_candidato.parquet", columns=load_cols_bens)

In [11]:
cadidato_resultado = pd.read_parquet(f"{in_path}resultado_candidatos.parquet")
candidatos = pd.read_parquet(f"{in_path}candidatos.parquet")

municipios = pd.read_parquet(f"{in_path}/municipios.parquet")
pib_municipios = pd.read_parquet(f"{in_path}municipios_pib.parquet")

cadidato_resultado['id_municipio_tse'] = cadidato_resultado['id_municipio_tse'].astype('str')
candidatos['id_municipio_tse'] = candidatos['id_municipio_tse'].astype('str')
pib_municipios['id_municipio'] = pib_municipios['id_municipio'].astype('str')

In [12]:
final_table = alziras.get_final_table(
    cadidato_resultado, candidatos, municipios, pib_municipios
)

## padroniza eleitos
final_table["resultado"] = (
    final_table["resultado"]
    .str.replace("eleito por media", "eleito")
    .str.replace("eleito por qp", "eleito")
)
## raca declarada
mask = final_table["raca"].notnull()
final_table = final_table[mask]

### Manipulate Tables

In [13]:
from scpts import alziras

receita_final, despesa_final, bens_final = alziras.get_financeiro_bens(
    final_table, bens, receita, despesa
)

### Gera Tabelas
-- --

**Definições**
 * Utilizei Negros como sendo pretos + pardos
 * Eleitos = eleito por media + eleito por qp
-- --
**Duvidas**
 * Gasto?
 * O que fazer com suplentes?
 -- --

### Receita total

In [14]:
group_cols = ["ano", "cargo", "genero", "raca", "resultado"]  # + ['sigla_uf']
select_cols = ["contagem", "valor_receita"]
atributo = None

receita_total, df_total = alziras.group_receita(
    receita_final, group_cols, select_cols, atributo
)
receita_total = receita_total[receita_total["resultado"] != "suplente"]

receita_total.to_csv(f"{out_path}receita_total.csv", index=False)


### Receita por atributo

In [15]:
group_cols = ["ano", "cargo", "genero", "raca", "resultado"]  # + ['sigla_uf']
select_cols = ["contagem", "valor_receita"]
atributo = "origem_receita"

receita_origem, df_total = alziras.group_receita(
    receita_final, group_cols, select_cols, atributo
)
receita_origem = receita_origem[receita_origem["resultado"] != "suplente"]
receita_origem.to_csv(f"{out_path}receita_por_origem.csv", index=False)

mask_proprio = receita_origem["origem_receita"] == "recursos proprios"
receita_origem[mask_proprio].to_csv(
    f"{out_path}receita_por_origem_proprio.csv", index=False
)

mask_pf = receita_origem["origem_receita"] == "recursos de pessoas fisicas"
receita_origem[mask_pf].to_csv(f"{out_path}receita_por_origem_pf.csv", index=False)

mask_crowdfunding = (
    receita_origem["origem_receita"] == "recursos de financiamento coletivo"
)
receita_origem[mask_crowdfunding].to_csv(
    f"{out_path}receita_por_origem_crowdfunding.csv", index=False
)

In [16]:
### Receita por fonte
group_cols = ["ano", "cargo", "genero", "raca", "resultado"]  # + ['sigla_uf']
select_cols = ["contagem", "valor_receita"]
atributo = "fonte_receita"

receita_fonte, df_total = alziras.group_receita(
    receita_final, group_cols, select_cols, atributo
)
receita_fonte = receita_fonte[receita_fonte["resultado"] != "suplente"]

receita_fonte.to_csv(f"{out_path}receita_por_fonte.csv", index=False)

### Patrimonio Total

In [17]:
group_cols = ["ano", "cargo", "genero", "raca", "resultado"]  # + ['sigla_uf']
select_cols = ["contagem", "patrimonio"]
atributo = None

patrimonio, df_total = alziras.group_receita(
    bens_final, group_cols, select_cols, atributo
)
patrimonio = patrimonio[patrimonio["resultado"] != "suplente"]

rename_cols = {
    "valor_receita": "patrimonio",
    "valor_receita_total": "patrimonio_total",
    "valor_receita_proporcional": "patrimonio_proporcional",
}
patrimonio = patrimonio.rename(columns=rename_cols)
patrimonio = patrimonio.drop(["valor_receita_per_capta"], 1)

patrimonio.to_csv(f"{out_path}patrimonio_total.csv", index=False)

### Despesa Total

In [18]:
group_cols = ["ano", "cargo", "genero", "raca", "resultado"]  # + ['sigla_uf']
select_cols = ["contagem", "valor_despesa"]
atributo = None

despesa_total, df_total = alziras.group_receita(
    despesa_final, group_cols, select_cols, atributo
)
despesa_total = despesa_total[despesa_total["resultado"] != "suplente"]

rename_cols = {
    "valor_receita": "valor_despesa",
    "valor_receita_total": "valor_despesa_total",
    "valor_receita_proporcional": "valor_despesa_proporcional",
    "valor_receita_per_capta": "valor_despesa_per_capta",
}
despesa_total = despesa_total.rename(columns=rename_cols)

despesa_total.to_csv(f"{out_path}despesa_total.csv", index=False)

# Demanda Adicional

In [9]:
cadidato_resultado = pd.read_parquet(f"{in_path}resultado_candidatos.parquet")

candidatos = pd.read_parquet(f"{in_path}candidatos.parquet")

municipios = pd.read_parquet(f"{in_path}/municipios.parquet")
pib_municipios = pd.read_parquet(f"{in_path}municipios_pib.parquet")

### load receita
load_cols = [
    "ano",
    "sigla_uf",
    "id_municipio_tse",
    "turno",
    "sequencial_candidato",
    "numero_candidato",
    "sigla_partido",
    "cargo",
    "origem_receita",
    "fonte_receita",
    "valor_receita",
]

receita = pd.read_parquet(f"{in_path}receita_candidatos.parquet", columns=load_cols)
mask = (receita["sigla_uf"] == "AM") & (receita["valor_receita"] > 10000000)
receita = receita[np.logical_not(mask)]

In [10]:
receita['id_municipio_tse'] = receita['id_municipio_tse'].astype(str)
cadidato_resultado['id_municipio_tse'] = cadidato_resultado['id_municipio_tse'].astype('str')
candidatos['id_municipio_tse'] = candidatos['id_municipio_tse'].astype('str')
pib_municipios['id_municipio'] = pib_municipios['id_municipio'].astype('str')

In [44]:
tabela_final = alziras.demanda_adicional(
    cadidato_resultado, 
    candidatos, 
    municipios, 
    pib_municipios, 
    receita
)

In [19]:
q_pop2020 = """
SELECT * FROM `basedosdados.br_ibge_populacao.municipio`
where (ano=2020)
"""

pop2020 = bd.read_sql(q_pop2020, billing_project_id="basedosdados-dev",from_file=True)
pop2020 = pop2020.rename(columns = {"populacao":"populacao_2020"})

Downloading: 100%|██████████| 5570/5570 [00:00<00:00, 8244.24rows/s]


In [45]:
tabela_final = tabela_final.merge(pop2020, on=['ano','id_municipio'], how='left')
tabela_final['populacao'] = np.where(tabela_final['ano']==2020, tabela_final['populacao_2020'], tabela_final['populacao'])
tabela_final = tabela_final.drop('populacao_2020',1)

tabela_final['populacao'] = tabela_final['populacao'].apply(lambda x: str(x).split('.')[0])
tabela_final['idade'] = tabela_final['idade'].apply(lambda x: str(x).split('.')[0])

tabela_final['populacao'] = tabela_final['populacao'].apply(lambda x: str(x).split('.')[0])
tabela_final['idade'] = tabela_final['idade'].apply(lambda x: str(x).split('.')[0])
tabela_final['id_candidato_bd'] = tabela_final['id_candidato_bd'].apply(lambda x: str(x).split('.')[0])
tabela_final['numero_partido'] = tabela_final['numero_partido'].apply(lambda x: str(x).split('.')[0])

In [49]:
tabela_final[['ano','valor_receita']].groupby(by='ano').sum()

,valor_receita
ano,
2016,2.995349e+09
2020,3.185731e+09


In [50]:
receita[['ano','valor_receita']].groupby(by='ano').sum()

,valor_receita
ano,
2016,3.030411e+09
2020,3.405879e+09


In [52]:
out_path

'../data/analise/mais/alziras/output/'

In [53]:
mask_prefeito = tabela_final['cargo']=='prefeito'
tabela_final[mask_prefeito].to_csv(f'{out_path}prefeitos_completo.csv', index=False)
tabela_final[np.logical_not(mask_prefeito)].to_csv(f'{out_path}vereadores_completo.csv', index=False)

In [26]:
tabela_final[mask_prefeito]

,ano,tipo_eleicao,turno,sigla_uf,id_municipio_tse,numero_candidato,id_candidato_bd,cargo,sigla_partido,resultado,votos,sequencial_candidato,cpf,nome_urna_candidato,numero_partido,situacao,genero,instrucao,idade,ocupacao,estado_civil,raca,faixa_etaria,id_municipio,PIB,populacao,municipio,regiao,pib_percapita,porte_municipal,decil_pib,reeleito_2020,nome_candidato,valor_receita
0,2016,eleicao ordinaria,1,AC,1007,11,799644.0,prefeito,PP,nao eleito,1468,10000002574,21711615234,Zilmar,11.0,deferido,feminino,ensino superior completo,49.0,professor de ensino medio,viuvo(a),parda,40 a 50 anos,1200138,159695867,9503.0,Bujari,Norte,16804.784489,0 a 20 mil,5.0,NaN,Maria Zilmar Da Rocha Paiva,53700.00
1,2016,eleicao ordinaria,1,AC,1007,13,768941.0,prefeito,PT,nao eleito,1145,10000002320,19763107253,Clovis Melo,13.0,deferido,masculino,ensino superior completo,54.0,ocupante de cargo em comissao,casado(a),parda,50 a 60 anos,1200138,159695867,9503.0,Bujari,Norte,16804.784489,0 a 20 mil,5.0,NaN,Clovis Alves De Melo E Silva,20713.70
2,2016,eleicao ordinaria,1,AC,1007,15,1208693.0,prefeito,MDB,nao eleito,1038,10000002447,46560246272,Francisco Bessa,15.0,deferido,masculino,ensino medio completo,41.0,gari ou lixeiro,casado(a),preta,40 a 50 anos,1200138,159695867,9503.0,Bujari,Norte,16804.784489,0 a 20 mil,5.0,NaN,Francisco Chagas De Souza Bessa,53122.40
3,2016,eleicao ordinaria,1,AC,1007,40,1281075.0,prefeito,PSB,nao eleito,700,10000001495,03051781215,Padeiro,40.0,indeferido,masculino,ensino fundamental incompleto,60.0,outros,casado(a),parda,50 a 60 anos,1200138,159695867,9503.0,Bujari,Norte,16804.784489,0 a 20 mil,5.0,n,João Edvaldo Teles De Lima,5029.30
4,2016,eleicao ordinaria,1,AC,1007,40,1281075.0,prefeito,PSB,nao eleito,700,10000003549,52275396268,Neto Padeiro,40.0,deferido,masculino,ensino medio completo,34.0,outros,solteiro(a),parda,30 a 40 anos,1200138,159695867,9503.0,Bujari,Norte,16804.784489,0 a 20 mil,5.0,NaN,José Vieira De Lima Neto,17001.50
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
967578,2020,eleicao ordinaria,1,TO,96970,15,1006269.0,prefeito,MDB,eleito,1734,270001151103,33264333104,Suzano,15.0,deferido,masculino,ensino superior completo,57.0,servidor publico estadual,casado(a),parda,50 a 60 anos,1701101,102279755,4741.0,Aparecida do Rio Negro,Norte,21573.456022,0 a 20 mil,6.0,n,Suzano Lino Marques,62800.00
967579,2020,eleicao ordinaria,1,TO,96970,65,1031946.0,prefeito,PC do B,nao eleito,11,270000841023,34723137149,Moizaniel Moiza,65.0,deferido,masculino,ensino superior incompleto,58.0,empresario,divorciado(a),parda,40 a 50 anos,1701101,102279755,4741.0,Aparecida do Rio Negro,Norte,21573.456022,0 a 20 mil,6.0,NaN,Moizaniel De Souza Silva,3000.00
967622,2020,eleicao ordinaria,1,TO,96997,25,1750387.0,prefeito,DEM,eleito,2555,270001107378,94664145187,Adriano Ribeiro,25.0,deferido,masculino,ensino superior completo,39.0,prefeito,casado(a),branca,50 a 60 anos,1703107,93732053,5614.0,Barrolândia,Norte,16696.126291,0 a 20 mil,5.0,s,Adriano José Ribeiro,66370.70
967652,2020,eleicao ordinaria,2,AM,2550,19,12051.0,prefeito,PODE,nao eleito,443747,40000793782,00164828249,Amazonino Mendes,19.0,deferido,masculino,ensino superior completo,81.0,advogado,viuvo(a),parda,60 mais,1302603,78192321274,2145444.0,Manaus,Norte,36445.752615,maior 500 mil,8.0,NaN,Amazonino Armando Mendes,3470000.00


### Bug

In [4]:
!pwd

/Users/m/gabinite_sv/bd+/mais_projects/notebooks


In [5]:
entrega_vereadores = pd.read_csv('../data/analise/mais/alziras/output/vereadores_completo.csv')


In [15]:
entrega_municipios = entrega_vereadores[['ano','sigla_uf','id_municipio','municipio','populacao']].drop_duplicates()

In [16]:
mask = (entrega_municipios['municipio'].str.contains('Assis Brasil')) | (entrega_municipios['municipio'].str.contains('Aracatu'))
entrega_municipios[mask]

,ano,id_municipio,sigla_uf,municipio,populacao
2017,2016,1200054,AC,Assis Brasil,6863.0
21669,2016,2902005,BA,Aracatu,14023.0
304557,2020,1200054,AC,Assis Brasil,7300.0
323747,2020,2902005,BA,Aracatu,13229.0


In [33]:
municipios = pd.read_parquet('../data/analise/mais/alziras/input/municipios.parquet')

vereadoras['id_municipio'] = vereadoras['id_municipio'].astype('str')
municipios['id_municipio'] = municipios['id_municipio'].astype('str')
entrega_vereadores['id_municipio'] = entrega_vereadores['id_municipio'].astype('str')

In [34]:
entrega_vereadores_2020 = vereadoras[vereadoras['ano']==2020]

In [40]:
mask = ~municipios['id_municipio'].isin(entrega_vereadores_2020['id_municipio'].unique())

municipios[mask].reset_index(drop=True)

,id_municipio,id_municipio_tse,municipio,regiao
0,1100304,132,Vilhena,Norte
1,1300060,2917,Amaturá,Norte
2,1301209,2259,Coari,Norte
3,1302603,2550,Manaus,Norte
4,1303908,2755,São Paulo de Olivença,Norte
5,1501709,4332,Bragança,Norte
6,1502152,4707,Canaã dos Carajás,Norte
7,1502301,4456,Capitão Poço,Norte
8,1502707,4537,Conceição do Araguaia,Norte
9,1502939,5975,Dom Eliseu,Norte


In [36]:
mask = ~municipios['id_municipio'].isin(entrega_vereadores_2020['id_municipio'].unique())


In [ ]:
def get_bug(df):
    mask = (df['id_municipio_tse']=='2550') & (df['ano']==2020)
    return df[mask]

In [ ]:
# cadidato_resultado = pd.read_parquet(f"{in_path}resultado_candidatos.parquet")
# candidatos = pd.read_parquet(f"{in_path}candidatos.parquet")
# municipios = pd.read_parquet(f"{in_path}/municipios.parquet")
# pib_municipios = pd.read_parquet(f"{in_path}municipios_pib.parquet")

In [ ]:
get_bug(cadidato_resultado)

In [ ]:
len(vereadoras['id_municipio'].unique().tolist())